<a href="https://colab.research.google.com/github/telmacarvalho/pi-4-univesp/blob/main/treinamento_ml_pi_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinamento do Modelo Multi-Target

## Importação de bibliotecas

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import json
import os

## Importação dos dados

In [2]:
# Autoriza acesso ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Extrai o dfs da pasta refined
file_path = '/content/drive/MyDrive/dados/csv/refined/data_processed.csv'
df_processed = pd.read_csv(file_path)
df_processed.head()

,cnae,total_acidentes,com_cat_registrada,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,sem_cat_registrada,ano,total_acidentes_seguinte,com_cat_registrada_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte,sem_cat_registrada_seguinte
0,910,1161,1104,900,141,63,57,2014,828,828,708,110,10,0
1,910,828,828,708,110,10,0,2015,562,525,454,56,15,37
2,910,562,525,454,56,15,37,2016,405,366,307,47,12,39
3,910,405,366,307,47,12,39,2017,517,479,408,45,26,38
4,910,517,479,408,45,26,38,2018,391,369,319,43,7,22


## 1. Divisão dos dados para treinamento

In [4]:
# Define as colunas atributos (total de acidentes, casos típicos, de trajeto e doenças ocupacionais)
colunas_alvo = [
    'total_acidentes',
    # 'com_cat_registrada',
    'acidentes_tipicos',
    'acidentes_trajeto',
    'doenca_trabalho'
    # 'sem_cat_registrada'
]

In [5]:
# Seleção dos Atributos (X)
X_data = df_processed[colunas_alvo]
X_data.head()

,total_acidentes,acidentes_tipicos,acidentes_trajeto,doenca_trabalho
0,1161,900,141,63
1,828,708,110,10
2,562,454,56,15
3,405,307,47,12
4,517,408,45,26


In [6]:
# Seleção dos Targets (y)
y_data = df_processed[[col + '_seguinte' for col in colunas_alvo]]
y_data.head()

,total_acidentes_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte
0,828,708,110,10
1,562,454,56,15
2,405,307,47,12
3,517,408,45,26
4,391,319,43,7


In [7]:
print("Dimensões de X (atributos):", X_data.shape)
print("Dimensões de y (alvos múltiplos):", y_data.shape)

Dimensões de X (atributos): (90, 4)
Dimensões de y (alvos múltiplos): (90, 4)


## Treinamento do modelo

In [8]:
df_processed.ano.unique()

array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [9]:
# Divisão temporal dos dados
train_indices = df_processed[df_processed['ano'] <= 2021].index
test_indices = df_processed[df_processed['ano'] == 2022].index

X_train, X_test = X_data.loc[train_indices], X_data.loc[test_indices]
y_train, y_test = y_data.loc[train_indices], y_data.loc[test_indices]

In [10]:
# Inicializa e treina o modelo
modelo_multi = RandomForestRegressor(n_estimators=100, random_state=42)
modelo_multi.fit(X_train, y_train)

print("Modelo de Regressão Multi-alvo treinado com sucesso!")

Modelo de Regressão Multi-alvo treinado com sucesso!


## 3. Avaliação do Modelo

In [11]:
X_test.columns

Index(['total_acidentes', 'acidentes_tipicos', 'acidentes_trajeto',
       'doenca_trabalho'],
      dtype='object')

In [12]:
X_test.shape

(10, 4)

*As previsões serão um array com 6 colunas*

In [13]:
# Faz as previsões
y_pred = modelo_multi.predict(X_test)
y_pred.shape

(10, 4)

In [14]:
# Cria um df com as previsões
df_pred = pd.DataFrame(y_pred, columns=y_test.columns, index=y_test.index)
df_pred.shape

(10, 4)

In [15]:
# Itera sobre cada coluna de alvo para calcular as métricas individualmente
metricas = {}
for i, col_name in enumerate(y_test.columns):
    mae = mean_absolute_error(y_test[col_name], df_pred[col_name])
    r2 = r2_score(y_test[col_name], df_pred[col_name])

    metricas[col_name.replace('_seguinte', '').upper()] = {
        'Erro Medio Absoluto (MAE)': mae,
        'Coeficiente R2': r2
    }

    print(f"--- {col_name.replace('_seguinte', '').upper()} ---")
    print(f"  Erro Médio Absoluto (MAE): {mae:.2f}")
    print(f"  Coeficiente R²: {r2:.2f}\n")

--- TOTAL_ACIDENTES ---
  Erro Médio Absoluto (MAE): 428.64
  Coeficiente R²: 0.83

--- ACIDENTES_TIPICOS ---
  Erro Médio Absoluto (MAE): 309.16
  Coeficiente R²: 0.84

--- ACIDENTES_TRAJETO ---
  Erro Médio Absoluto (MAE): 75.16
  Coeficiente R²: 0.87

--- DOENCA_TRABALHO ---
  Erro Médio Absoluto (MAE): 14.72
  Coeficiente R²: 0.61



In [16]:
# Busca as colunas 'cnae' e 'ano' do DataFrame original usando o índice do conjunto de teste
identificadores = df_processed.loc[y_test.index, ['cnae', 'ano']]
identificadores.head()

,cnae,ano
8,910,2022
17,1921,2022
26,2513,2022
35,3321,2022
44,3520,2022


In [17]:
df_pred.head(2)

,total_acidentes_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte
8,321.45,241.32,29.07,24.96
17,142.80,76.70,31.84,5.78


In [18]:
# Converte colunas para int
df_pred = df_pred.astype(int)
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 8 to 89
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   total_acidentes_seguinte    10 non-null     int64
 1   acidentes_tipicos_seguinte  10 non-null     int64
 2   acidentes_trajeto_seguinte  10 non-null     int64
 3   doenca_trabalho_seguinte    10 non-null     int64
dtypes: int64(4)
memory usage: 400.0 bytes


In [19]:
# Renomea as colunas de previsão
y_pred_renomeado = df_pred.rename(columns=lambda c: c.replace('_seguinte', '_previsto'))
y_pred_renomeado.columns

Index(['total_acidentes_previsto', 'acidentes_tipicos_previsto',
       'acidentes_trajeto_previsto', 'doenca_trabalho_previsto'],
      dtype='object')

In [20]:
# Junta tudo em um único df de resultados
df_resultados_completos = pd.concat([identificadores, y_test, y_pred_renomeado], axis=1)
df_resultados_completos.head(2)

,cnae,ano,total_acidentes_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte,total_acidentes_previsto,acidentes_tipicos_previsto,acidentes_trajeto_previsto,doenca_trabalho_previsto
8,910,2022,490,357,62,31,321,241,29,24
17,1921,2022,265,111,72,22,142,76,31,5


In [21]:
df_resultados_completos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 8 to 89
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   cnae                        10 non-null     int64
 1   ano                         10 non-null     int64
 2   total_acidentes_seguinte    10 non-null     int64
 3   acidentes_tipicos_seguinte  10 non-null     int64
 4   acidentes_trajeto_seguinte  10 non-null     int64
 5   doenca_trabalho_seguinte    10 non-null     int64
 6   total_acidentes_previsto    10 non-null     int64
 7   acidentes_tipicos_previsto  10 non-null     int64
 8   acidentes_trajeto_previsto  10 non-null     int64
 9   doenca_trabalho_previsto    10 non-null     int64
dtypes: int64(10)
memory usage: 880.0 bytes


In [22]:
# Renomea as colunas
df_resultados_completos = df_resultados_completos.rename(columns=lambda c: c.replace('_seguinte', ''))
df_resultados_completos.columns

Index(['cnae', 'ano', 'total_acidentes', 'acidentes_tipicos',
       'acidentes_trajeto', 'doenca_trabalho', 'total_acidentes_previsto',
       'acidentes_tipicos_previsto', 'acidentes_trajeto_previsto',
       'doenca_trabalho_previsto'],
      dtype='object')

In [23]:
# Adequa o ano
df_resultados_completos['ano'] = df_resultados_completos['ano'].replace(2022,2023)
df_resultados_completos.head()

,cnae,ano,total_acidentes,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,total_acidentes_previsto,acidentes_tipicos_previsto,acidentes_trajeto_previsto,doenca_trabalho_previsto
8,910,2023,490,357,62,31,321,241,29,24
17,1921,2023,265,111,72,22,142,76,31,5
26,2513,2023,459,378,51,5,521,397,75,8
35,3321,2023,897,698,110,5,679,503,82,11
44,3520,2023,56,27,23,0,39,22,9,0


In [24]:
df_resultados_completos.shape

(10, 10)

In [26]:
# df_resultados_completos = df_resultados_completos.iloc[:,[0,1,2,8,3,9,4,10,5,11,6,12,7,13]]
df_resultados_completos = df_resultados_completos.iloc[:,[0,1,2,6,3,7,4,8,5,9]]
df_resultados_completos.head()

,cnae,ano,total_acidentes,total_acidentes_previsto,acidentes_tipicos,acidentes_tipicos_previsto,acidentes_trajeto,acidentes_trajeto_previsto,doenca_trabalho,doenca_trabalho_previsto
8,910,2023,490,321,357,241,62,29,31,24
17,1921,2023,265,142,111,76,72,31,22,5
26,2513,2023,459,521,378,397,51,75,5,8
35,3321,2023,897,679,698,503,110,82,5,11
44,3520,2023,56,39,27,22,23,9,0,0


## 4. Amazenamento no drive

In [27]:
# Define o caminho e o nome da pasta no Google Drive
caminho_drive = '/content/drive/MyDrive/dados/output/'

In [28]:
# Salva o Modelo Treinado
caminho_modelo = os.path.join(caminho_drive, 'modelo_random_forest_multi.joblib')
joblib.dump(modelo_multi, caminho_modelo)

print(f"Modelo salvo com sucesso em: {caminho_modelo}")

Modelo salvo com sucesso em: /content/drive/MyDrive/dados/output/modelo_random_forest_multi.joblib


In [29]:
# Salva o dicionário de métricas em JSON
caminho_metricas = os.path.join(caminho_drive, 'metricas_modelo.json')
with open(caminho_metricas, 'w') as f:
    json.dump(metricas, f, indent=4) # indent=4 para deixar o arquivo bem formatado

print(f"Métricas salvas com sucesso em: {caminho_metricas}")

Métricas salvas com sucesso em: /content/drive/MyDrive/dados/output/metricas_modelo.json


In [30]:
# Salva as previsões
file_path = f'{caminho_drive}results_test.csv'
df_resultados_completos.to_csv(file_path, index=False)

print(f"Arquivo foi salvo com sucesso em: {file_path}")

Arquivo foi salvo com sucesso em: /content/drive/MyDrive/dados/output/results_test.csv
